# ういちと俺たちの夏休み -自由研究-

## 事前準備

### 画像読み込み

In [ ]:
from tkinter import filedialog
import os

# ファイルをダイアログから選択(PNG形式のみ)
source_filepath = filedialog.askopenfilename(initialdir = "./", filetypes = [("Image File","*.png")])
print(source_filepath)

# 作業用にディレクトリ名を画像保存先をパスを取得しておく
work_dir = os.path.dirname(source_filepath)


In [ ]:
from IPython.display import Image as display_img

display_img(source_filepath)

### PNGだと扱いづらいので非圧縮BMPに変換

カラーパレット情報は不要なので24bit BPMに変換 <br>
デコーダを書いてもいいんだけど、完全に蛇足なので素直にライブラリを使用 <br>
え？最初から最後まで蛇足だって？そんなわけ...

In [ ]:
from PIL import Image

# 保存先変数
bmp_filename = "img.bmp"
bmp_path = os.path.join(work_dir, bmp_filename)

# 画像読み込み
png_img = Image.open(source_filepath)

# 透過情報を無視するため、一度RGBに変換
rgb_img = png_img.convert("RGB")

# 24bit BMP変換
rgb_img.save(bmp_path, format="BMP")

## グレースケール

ピクセル当たりの情報量を1/3に減らすため、グレースケールにする

In [ ]:
import struct

# BMPをバイナリ読み込み
with open(bmp_path, 'rb') as f:
    # BMPヘッダ
    header = f.read(14)
    # DIBヘッダ (BITMAPINFOHEADER)
    dib_header = f.read(40)

    # ヘッダから情報を取得
    width, height, planes, bit_count = struct.unpack("<iiHH", dib_header[4:16])

    if bit_count != 24:
        raise ValueError("このコードは24bit BMP専用です。")

    # 1行のピクセルに必要なバイト数（4バイト境界に揃えるパディング込み）
    row_padded = (width * 3 + 3) & ~3

    # 出力ファイル名定義
    grayscale_bmp_filename = "gray_scale.bmp"
    grayscale_bmp_path = os.path.join(work_dir, grayscale_bmp_filename)

    # 出力ファイルを書き込み開始
    with open(grayscale_bmp_path, "wb") as out:
        out.write(header)
        out.write(dib_header)

        # ピクセルデータを1行ごとに処理
        for _ in range(height):
            row = bytearray(f.read(row_padded))  # 1行分読み込み
            for x in range(0, width * 3, 3):
                b, g, r = row[x], row[x+1], row[x+2]
                gray = int(0.114*b + 0.587*g + 0.299*r)
                row[x] = row[x+1] = row[x+2] = gray
            out.write(row)

### 現在のイメージ確認

In [ ]:
# グレースケール画像の確認(BMP->PNG)
grayscale_png_filename = "gray_scale.png"
grayscale_png_path = os.path.join(work_dir, grayscale_png_filename)
g_bmp_img = Image.open(grayscale_bmp_path)
g_bmp_img.save(grayscale_png_path, format="PNG")

# 画像表示
display_img(grayscale_png_path)


## ノイズ除去（ガウシアンフィルタ）

In [ ]:
import math

# 3*3ガウシアンカーネル
kernel = [
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
]
WEIGHT_SUM = 16

# BMPをバイナリ読み込み
with open(grayscale_bmp_path, 'rb') as input:
    # BMPヘッダ
    header = input.read(14)
    # DIBヘッダ (BITMAPINFOHEADER)
    dib_header = input.read(40)

    # ヘッダから情報を取得
    width, height, planes, bit_count = struct.unpack("<iiHH", dib_header[4:16])

    # 1行のピクセルに必要なバイト数（4バイト境界に揃えるパディング込み）
    row_padded = (width * 3 + 3) & ~3

    # 出力ファイル名定義
    gaussian_bmp_filename = "gaussian.bmp"
    gaussian_bmp_path = os.path.join(work_dir, gaussian_bmp_filename)

    # 出力ファイルを書き込み開始
    with open(gaussian_bmp_path, "wb") as output:
        output.write(header)
        output.write(dib_header)
        
        # 全体を読み込むとメモリが不足するため3行ずつ読み込み
        # 先頭行を読み込み
        first_row = input.read(row_padded)
        lum_first = bytearray(width)
        height_absolute = abs(height)
        for w in range(width):
            b = first_row[w*3]
            g = first_row[w*3+1]
            r = first_row[w*3+2]
            lum_first[w] = int(0.114*b + 0.587*g + 0.299*r)

        # 2行目を読み込み
        second_row = input.read(row_padded)
        lum_second = bytearray(width)
        for w in range(width):
            b = second_row[w*3]
            g = second_row[w*3+1]
            r = second_row[w*3+2]
            lum_second[w] = int(0.114*b + 0.587*g + 0.299*r)

        # 1行目はそのままでは処理できないため1行目を複製して使用する
        out_row = bytearray(row_padded)
        for w in range(width):
            # 左右の端は0として処理
            left = w-1 if w-1>=0 else 0
            right = w+1 if w+1<width else width-1
            s = (
                lum_first[left]*kernel[0][0] + lum_first[w]*kernel[0][1] + lum_first[right]*kernel[0][2] +
                lum_first[left]*kernel[1][0] + lum_first[w]*kernel[1][1] + lum_first[right]*kernel[1][2] +
                lum_second[left]*kernel[2][0] + lum_second[w]*kernel[2][1] + lum_second[right]*kernel[2][2]
            )
            g = (s + WEIGHT_SUM//2)//WEIGHT_SUM
            off = w*3
            out_row[off] = out_row[off+1] = out_row[off+2] = g
        output.write(out_row)

        prev = lum_first
        curr = lum_second

        # 2行目以降を処理
        for h in range(1, height_absolute):
            if h < height_absolute-1:
                next_row = input.read(row_padded)
                next_lum = bytearray(width)
                for w in range(width):
                    b = next_row[w*3]
                    g = next_row[w*3+1]
                    r = next_row[w*3+2]
                    next_lum[w] = int(0.114*b + 0.587*g + 0.299*r)
            else:
                # 最終行はそのままでは処理できないため最終行を複製して使用する
                next_lum = curr

            out_row = bytearray(row_padded)
            for w in range(width):
                left = w-1 if w-1>=0 else 0
                right = w+1 if w+1<width else width-1
                s = (
                    prev[left]*kernel[0][0] + prev[w]*kernel[0][1] + prev[right]*kernel[0][2] +
                    curr[left]*kernel[1][0] + curr[w]*kernel[1][1] + curr[right]*kernel[1][2] +
                    next_lum[left]*kernel[2][0] + next_lum[w]*kernel[2][1] + next_lum[right]*kernel[2][2]
                )
                g = (s + WEIGHT_SUM//2)//WEIGHT_SUM
                off = w*3
                # グレースケールのためすべて同じ値をセット
                out_row[off] = out_row[off+1] = out_row[off+2] = g
            output.write(out_row)

            prev, curr = curr, next_lum


### 現在のイメージ確認

In [ ]:
# ノイズ除去後画像の確認(BMP->PNG)
gaussian_png_filename = "gaussian.png"
gaussian_png_path = os.path.join(work_dir, gaussian_png_filename)
gaus_bmp_img = Image.open(grayscale_bmp_path)
gaus_bmp_img.save(gaussian_png_path, format="PNG")

# 画像表示
display_img(gaussian_png_path)


## エッジ抽出 (ソーベルカーネル)

In [ ]:
# 横方向カーネル
Kx = [[-1,0,1],[-2,0,2],[-1,0,1]]
# 縦方向カーネル
Ky = [[-1,-2,-1],[0,0,0],[1,2,1]]

with open(gaussian_bmp_path,"rb") as input:
    header = input.read(14)
    dib = input.read(40)
    width, height, planes, bit_count = struct.unpack("<iiHH", dib[4:16])
    if bit_count != 24:
        raise ValueError("24bit BMP 専用")
    height_abs = abs(height)
    row_padded = (width*3 +3) & ~3

    # 出力ファイル名定義
    sobal_bmp_filename = "sobal.bmp"
    sobal_bmp_path = os.path.join(work_dir, sobal_bmp_filename)


    with open(sobal_bmp_path,"wb") as output:
        output.write(header)
        output.write(dib)

        # 先頭行を読み込み
        first_row = input.read(row_padded)
        lum_prev = bytearray(width)
        for w in range(width):
            b,g,r = first_row[w*3], first_row[w*3+1], first_row[w*3+2]
            lum_prev[w] = int(0.114*b + 0.587*g + 0.299*r)

        # 2行目を読み込み
        second_row = input.read(row_padded)
        lum_curr = bytearray(width)
        for w in range(width):
            b,g,r = second_row[w*3], second_row[w*3+1], second_row[w*3+2]
            lum_curr[w] = int(0.114*b + 0.587*g + 0.299*r)

        # 1行目はそのままでは処理できないため1行目を複製して使用する
        out_row = bytearray(row_padded)
        for w in range(width):
            left, right = w-1 if w>0 else 0, w+1 if w<width-1 else width-1
            Gx = Kx[0][0]*lum_prev[left]+Kx[0][1]*lum_prev[w]+Kx[0][2]*lum_prev[right]+\
                Kx[1][0]*lum_prev[left]+Kx[1][1]*lum_prev[w]+Kx[1][2]*lum_prev[right]+\
                Kx[2][0]*lum_curr[left]+Kx[2][1]*lum_curr[w]+Kx[2][2]*lum_curr[right]
            Gy = Ky[0][0]*lum_prev[left]+Ky[0][1]*lum_prev[w]+Ky[0][2]*lum_prev[right]+\
                Ky[1][0]*lum_prev[left]+Ky[1][1]*lum_prev[w]+Ky[1][2]*lum_prev[right]+\
                Ky[2][0]*lum_curr[left]+Ky[2][1]*lum_curr[w]+Ky[2][2]*lum_curr[right]
            g = min(255, int(math.sqrt(Gx*Gx + Gy*Gy)))
            off = w*3
            out_row[off] = out_row[off+1] = out_row[off+2] = g
        output.write(out_row)

        prev, curr = lum_prev, lum_curr

        # 2行目以降を処理
        for h in range(1,height_abs):
            if h < height_abs-1:
                next_row = input.read(row_padded)
                lum_next = bytearray(width)
                for w in range(width):
                    b,g,r = next_row[w*3], next_row[w*3+1], next_row[w*3+2]
                    lum_next[w] = int(0.114*b + 0.587*g + 0.299*r)
            else:
                # 最終行はそのままでは処理できないため最終行を複製して使用する
                lum_next = curr

            out_row = bytearray(row_padded)
            for w in range(width):
                left, right = w-1 if w>0 else 0, w+1 if w<width-1 else width-1
                Gx = Kx[0][0]*prev[left]+Kx[0][1]*prev[w]+Kx[0][2]*prev[right]+\
                    Kx[1][0]*curr[left]+Kx[1][1]*curr[w]+Kx[1][2]*curr[right]+\
                    Kx[2][0]*lum_next[left]+Kx[2][1]*lum_next[w]+Kx[2][2]*lum_next[right]
                Gy = Ky[0][0]*prev[left]+Ky[0][1]*prev[w]+Ky[0][2]*prev[right]+\
                    Ky[1][0]*curr[left]+Ky[1][1]*curr[w]+Ky[1][2]*curr[right]+\
                    Ky[2][0]*lum_next[left]+Ky[2][1]*lum_next[w]+Ky[2][2]*lum_next[right]

                g = min(255, int(math.sqrt(Gx*Gx + Gy*Gy)))
                off = w*3
                # グレースケールのためすべて同じ値をセット
                out_row[off] = out_row[off+1] = out_row[off+2] = g

            output.write(out_row)
            prev, curr = curr, lum_next

### 現在のイメージ確認

In [ ]:
# エッジ抽出後画像の確認(BMP->PNG)
sobal_png_filename = "sobal.png"
sobal_png_path = os.path.join(work_dir, sobal_png_filename)
sobal_bmp_img = Image.open(sobal_bmp_path)
sobal_bmp_img.save(sobal_png_path, format="PNG")

# 画像表示
display_img(sobal_png_path)


## 非極大値抑制処理 (NMS)

In [ ]:
# --- BMP読み込み ---
with open(sobal_bmp_path, "rb") as fin:
    header = fin.read(14)  # BMPヘッダ
    dib = fin.read(40)     # DIBヘッダ

    width, height, planes, bit_count = struct.unpack("<iiHH", dib[4:16])
    if bit_count != 24:
        raise ValueError("24bit BMP 専用")

    height_abs = abs(height)
    row_padded = (width*3 + 3) & ~3
    pixel_data_offset = 14 + 40

    # --- 出力ファイル作成（全体サイズを予約） ---
    nms_bmp_path = os.path.join(work_dir, "nms.bmp")
    with open(nms_bmp_path, "wb") as fout:
        fout.write(header)
        fout.write(dib)
        # BMPサイズ分をゼロで埋める
        fout.write(b'\x00' * (row_padded * height_abs))

    # --- 出力ファイルをランダムアクセスで開く ---
    with open(nms_bmp_path, "r+b") as fout:
        # 行オフセット計算
        def get_row_offset(row_index):
            if height > 0:
                # 下から上に並ぶ
                return pixel_data_offset + (height_abs - 1 - row_index) * row_padded
            else:
                # 上から下に並ぶ
                return pixel_data_offset + row_index * row_padded

        # --- 最初の2行読み込み ---
        first_row_bytes = fin.read(row_padded)
        prev = bytearray([first_row_bytes[w*3] for w in range(width)])

        second_row_bytes = fin.read(row_padded)
        curr = bytearray([second_row_bytes[w*3] for w in range(width)])

        for h in range(1, height_abs):
            if h < height_abs - 1:
                next_row_bytes = fin.read(row_padded)
                next_buf = bytearray([next_row_bytes[w*3] for w in range(width)])
            else:
                next_buf = curr  # 最下行は自己複製

            out_row = bytearray(row_padded)
            for w in range(width):
                left = max(0, w-1)
                right = min(width-1, w+1)

                dx = int(curr[right]) - int(curr[left])
                dy = int(next_buf[w]) - int(prev[w])
                angle = math.degrees(math.atan2(dy, dx)) % 180

                if (0 <= angle < 22.5) or (157.5 <= angle < 180):
                    direction = 0
                elif 22.5 <= angle < 67.5:
                    direction = 45
                elif 67.5 <= angle < 112.5:
                    direction = 90
                else:
                    direction = 135

                g = curr[w]
                if direction == 0:
                    g1, g2 = curr[left], curr[right]
                elif direction == 90:
                    g1, g2 = prev[w], next_buf[w]
                elif direction == 45:
                    g1, g2 = prev[right], next_buf[left]
                else:  # 135
                    g1, g2 = prev[left], next_buf[right]

                val = g if g >= g1 and g >= g2 else 0
                off = w*3
                out_row[off] = out_row[off+1] = out_row[off+2] = val

            # --- 正しい行オフセットに書き込み ---
            fout.seek(get_row_offset(h-1))
            fout.write(out_row)

            prev, curr = curr, next_buf


with open(nms_bmp_path, "r+b") as f:
    # --- ヘッダ読み込み ---
    header = f.read(14)
    dib = f.read(40)
    width, height, planes, bit_count = struct.unpack("<iiHH", dib[4:16])

    if bit_count != 24:
        raise ValueError("24bit BMP専用")

    height_abs = abs(height)
    row_padded = (width * 3 + 3) & ~3
    pixel_data_offset = 14 + 40

    # --- 行バッファを用意 ---
    # 2行ずつ入れ替えながら上下反転する場合
    for i in range(height_abs // 2):
        top_offset = pixel_data_offset + i * row_padded
        bottom_offset = pixel_data_offset + (height_abs - 1 - i) * row_padded

        # 上下行を読み込む
        f.seek(top_offset)
        top_row = bytearray(f.read(row_padded))

        f.seek(bottom_offset)
        bottom_row = bytearray(f.read(row_padded))

        # 行を入れ替えて書き戻す
        f.seek(top_offset)
        f.write(bottom_row)

        f.seek(bottom_offset)
        f.write(top_row)

### 現在のイメージ確認

In [ ]:
# 非極大値抑制処理後画像の確認(BMP->PNG)
nms_png_filename = "nms.png"
nms_png_path = os.path.join(work_dir, nms_png_filename)
nms_bmp_img = Image.open(nms_bmp_path)
nms_bmp_img.save(nms_png_path, format="PNG")

# 画像表示
display_img(nms_png_path)


## 閾値処理

In [ ]:
# ヒステリシス閾値
high_thresh = 140
low_thresh = 20

with open(nms_bmp_path, "rb") as fin:
    # --- ヘッダ読み込み ---
    header = fin.read(14)
    dib = fin.read(40)
    width, height, planes, bit_count = struct.unpack("<iiHH", dib[4:16])

    if bit_count != 24:
        raise ValueError("24bit BMP専用")

    height_abs = abs(height)
    row_padded = (width * 3 + 3) & ~3
    pixel_data_offset = 14 + 40

    # --- 出力ファイル準備 ---
    his_bmp_path = os.path.join(work_dir, "hysteresis.bmp")
    with open(his_bmp_path, "wb") as fout:
        fout.write(header)
        fout.write(dib)
        fout.write(b'\x00' * (row_padded * height_abs))

    # --- 出力ファイルをランダムアクセスで開く ---
    with open(his_bmp_path, "r+b") as fout:
        def get_row_offset(row_index):
            if height > 0:
                # 下から上に並ぶ
                return pixel_data_offset + (height_abs - 1 - row_index) * row_padded
            else:
                # 上から下に並ぶ
                return pixel_data_offset + row_index * row_padded

        # --- 最初の2行読み込み ---
        fin.seek(get_row_offset(0))
        prev_row_bytes = fin.read(row_padded)
        prev = bytearray([prev_row_bytes[w*3] for w in range(width)])

        fin.seek(get_row_offset(1))
        curr_row_bytes = fin.read(row_padded)
        curr = bytearray([curr_row_bytes[w*3] for w in range(width)])

        for h in range(1, height_abs):
            # 次行読み込み
            if h < height_abs - 1:
                fin.seek(get_row_offset(h+1))
                next_row_bytes = fin.read(row_padded)
                next_buf = bytearray([next_row_bytes[w*3] for w in range(width)])
            else:
                next_buf = curr  # 最下行は自己複製

            # --- ヒステリシス処理 ---
            out_row = bytearray(row_padded)
            for w in range(width):
                val = curr[w]
                if val >= high_thresh:
                    out_val = 255
                elif val >= low_thresh:
                    neighbors = [
                        prev[max(0, w-1)], prev[w], prev[min(width-1, w+1)],
                        curr[max(0, w-1)], curr[min(width-1, w+1)],
                        next_buf[max(0, w-1)], next_buf[w], next_buf[min(width-1, w+1)]
                    ]
                    out_val = 255 if any(n >= high_thresh for n in neighbors) else 0
                else:
                    out_val = 0

                off = w*3
                out_row[off] = out_row[off+1] = out_row[off+2] = out_val

            # --- 出力ファイルに書き込み ---
            fout.seek(get_row_offset(h))
            fout.write(out_row)

            prev, curr = curr, next_buf



### 現在のイメージ確認

In [ ]:
# ヒステリシス閾値処理後画像の確認(BMP->PNG)
hysteresis_png_filename = "hysteresis.png"
hysteresis_png_path = os.path.join(work_dir, hysteresis_png_filename)
hysteresis_bmp_img = Image.open(his_bmp_path)
hysteresis_bmp_img.save(hysteresis_png_path, format="PNG")

# 画像表示
display_img(hysteresis_png_path)
